In [1]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from numpy import exp
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (8,6)
from sklearn.model_selection import train_test_split
import pylab 
import time

In [3]:
df_loc = pd.read_csv("../synthetic_data_simulations/2d_biv_nonStationary_1200.csv", sep = ",")
N = len(df_loc)

s = np.vstack((df_loc["x"],df_loc["y"])).T
y = np.array(df_loc[["var1","var2"]])
### Basis functions

num_basis = [2**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
#knots_1d = [np.linspace(0,1,i) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

In [4]:
# model layers 

input_dim = Input(shape = (phi.shape[1], ))
layer1 = Dense(100, kernel_initializer='he_uniform', activation = 'relu')(input_dim)
layer2 = Dense(100, activation = 'relu')(layer1)
layer3 = Dense(100, activation = 'relu')(layer2)
layer4 = Dense(50, activation = 'relu')(layer3)
layer5 = Dense(50, activation = 'relu')(layer4)
final_layer = Dense(2, activation = 'linear')(layer5)

In [5]:
s_train, s_test, encoder_train, encoder_test    , y_train, y_test= train_test_split(s, phi, y, 
                                                                                    test_size=0.2)
print(s_test.shape)
s_train_ensemble, s_train_mse, X_train_ensemble, X_train_mse, y_train_ensemble, y_train_mse= train_test_split(s_train, encoder_train, y_train, test_size=0.2)
print(s_train_ensemble.shape)

(240, 2)
(768, 2)


In [6]:
s_train_ensemble1, s_train_ensemble2, X_train_ensemble1, X_train_ensemble2, y_train_ensemble1, y_train_ensemble2= train_test_split(s_train_ensemble, X_train_ensemble, y_train_ensemble, test_size=0.2)
print(s_train_ensemble1.shape)
base_model = Model(inputs = input_dim, outputs = final_layer)
optimizer = keras.optimizers.Adam(lr=0.01)
# Compile the Model
base_model.compile(optimizer = optimizer, loss = 'mse')
base_model.summary()
base_model.fit(X_train_ensemble1, y_train_ensemble1,validation_split = 0.1, epochs = 400,
                batch_size = 64,verbose = 2)

(614, 2)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 110)]             0         
_________________________________________________________________
dense (Dense)                (None, 100)               11100     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 2)              

9/9 - 0s - loss: 3.5284e-04 - val_loss: 0.0679
Epoch 121/400
9/9 - 0s - loss: 2.5978e-04 - val_loss: 0.0688
Epoch 122/400
9/9 - 0s - loss: 2.0388e-04 - val_loss: 0.0689
Epoch 123/400
9/9 - 0s - loss: 2.7323e-04 - val_loss: 0.0695
Epoch 124/400
9/9 - 0s - loss: 2.8168e-04 - val_loss: 0.0687
Epoch 125/400
9/9 - 0s - loss: 2.0999e-04 - val_loss: 0.0667
Epoch 126/400
9/9 - 0s - loss: 2.1458e-04 - val_loss: 0.0688
Epoch 127/400
9/9 - 0s - loss: 2.3184e-04 - val_loss: 0.0657
Epoch 128/400
9/9 - 0s - loss: 2.3296e-04 - val_loss: 0.0694
Epoch 129/400
9/9 - 0s - loss: 1.3088e-04 - val_loss: 0.0664
Epoch 130/400
9/9 - 0s - loss: 2.1681e-04 - val_loss: 0.0687
Epoch 131/400
9/9 - 0s - loss: 1.8245e-04 - val_loss: 0.0684
Epoch 132/400
9/9 - 0s - loss: 1.5860e-04 - val_loss: 0.0673
Epoch 133/400
9/9 - 0s - loss: 1.0684e-04 - val_loss: 0.0666
Epoch 134/400
9/9 - 0s - loss: 1.2158e-04 - val_loss: 0.0683
Epoch 135/400
9/9 - 0s - loss: 1.1617e-04 - val_loss: 0.0680
Epoch 136/400
9/9 - 0s - loss: 8.7510e

9/9 - 0s - loss: 1.3412e-04 - val_loss: 0.0708
Epoch 257/400
9/9 - 0s - loss: 1.6243e-04 - val_loss: 0.0660
Epoch 258/400
9/9 - 0s - loss: 1.6161e-04 - val_loss: 0.0733
Epoch 259/400
9/9 - 0s - loss: 2.1960e-04 - val_loss: 0.0648
Epoch 260/400
9/9 - 0s - loss: 2.4243e-04 - val_loss: 0.0722
Epoch 261/400
9/9 - 0s - loss: 2.5541e-04 - val_loss: 0.0700
Epoch 262/400
9/9 - 0s - loss: 1.7298e-04 - val_loss: 0.0680
Epoch 263/400
9/9 - 0s - loss: 2.2468e-04 - val_loss: 0.0664
Epoch 264/400
9/9 - 0s - loss: 2.6212e-04 - val_loss: 0.0703
Epoch 265/400
9/9 - 0s - loss: 3.2452e-04 - val_loss: 0.0654
Epoch 266/400
9/9 - 0s - loss: 3.4440e-04 - val_loss: 0.0678
Epoch 267/400
9/9 - 0s - loss: 2.6370e-04 - val_loss: 0.0692
Epoch 268/400
9/9 - 0s - loss: 2.0205e-04 - val_loss: 0.0684
Epoch 269/400
9/9 - 0s - loss: 1.6671e-04 - val_loss: 0.0678
Epoch 270/400
9/9 - 0s - loss: 1.4204e-04 - val_loss: 0.0671
Epoch 271/400
9/9 - 0s - loss: 1.5911e-04 - val_loss: 0.0701
Epoch 272/400
9/9 - 0s - loss: 1.4976e

Epoch 392/400
9/9 - 0s - loss: 1.8602e-04 - val_loss: 0.0677
Epoch 393/400
9/9 - 0s - loss: 2.6355e-04 - val_loss: 0.0680
Epoch 394/400
9/9 - 0s - loss: 5.1859e-04 - val_loss: 0.0674
Epoch 395/400
9/9 - 0s - loss: 5.7980e-04 - val_loss: 0.0679
Epoch 396/400
9/9 - 0s - loss: 9.2773e-04 - val_loss: 0.0714
Epoch 397/400
9/9 - 0s - loss: 9.3188e-04 - val_loss: 0.0703
Epoch 398/400
9/9 - 1s - loss: 0.0014 - val_loss: 0.0596
Epoch 399/400
9/9 - 0s - loss: 0.0026 - val_loss: 0.0563
Epoch 400/400
9/9 - 0s - loss: 0.0027 - val_loss: 0.0767


In [6]:
base_model1 = Model(inputs = input_dim, outputs = layer3)

In [7]:
def mse(y_pred,y_true):
    mse = np.mean((y_pred-y_true)**2)
    return mse

def mae(y_pred,y_true):
    mae = np.mean(np.absolute(y_pred-y_true))
    return mae
# define and fit the model
def fit_model(X_train, y_train,X_test,y_test):
    # DeepKriging model for continuous data
    ensemble_model = Sequential()

    ensemble_model.add(base_model1)
#     ensemble_model.add(Dense(50, activation = "relu"))
    ensemble_model.add(Dense(50, activation = "relu"))
    ensemble_model.add(Dense(2, activation='linear'))
    ensemble_model.layers[-3].trainable = False
    optimizer = keras.optimizers.Adam(lr=0.01)
    ensemble_model.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])
    ensemble_model.fit(X_train, y_train,
                       validation_data=(X_test,y_test), epochs = 200, batch_size = 64, verbose = 0)
    return ensemble_model

# fit an ensemble of models
def fit_ensemble(n_members, X_train, Y_train):
    ensemble = list()
    for i in range(n_members):
        x_train, x_test,y_train,y_test = train_test_split(X_train,Y_train,test_size = 0.2)
        # define and fit the model on the training set
        model = fit_model(x_train, y_train,x_test,y_test)
        # evaluate model on the test set
        yhat = model.predict(x_test, verbose=0)
        mae1 = mae(yhat, y_test)
        print('>%d, MAE: %.3f' % (i+1, mae1))
        # store the model
        ensemble.append(model)
    return ensemble
def y_list_uni(yhat,i):
    the_list = list()
    for j in range(len(yhat)):
        the_list.append(yhat[j][0][i])
    return the_list
# make predictions with the ensemble and calculate a prediction interval
def variance(data):
    # Number of observations
    n = len(data)
    # Mean of the data
    mean = sum(data) / n
    # Square deviations
    deviations = [(x - mean) ** 2 for x in data]
    # Variance
    variance = sum(deviations) / (n-1)
    return variance
def predict_with_pi(ensemble, X):
    # make predictions
    quantiles1 = []
    quantiles2 = []
    yhat = [model.predict(X, verbose=0) for model in ensemble]
    yhat=np.asarray(yhat)
    
    var1_pred = np.asarray(y_list_uni(yhat,0))
    var2_pred = np.asarray(y_list_uni(yhat,1))
    probabilities = [0.025,0.975]
    for i in range(len(probabilities)):
        quantiles1.append(np.quantile(var1_pred,probabilities[i]))
        quantiles2.append(np.quantile(var2_pred,probabilities[i]))
#     y_list_uni = 
    # calculate 95% gaussian prediction interval
    mean1 = var1_pred.mean()
    var1 = variance(var1_pred)
    mean2 = var2_pred.mean()
    var2 = variance(var2_pred)
    return mean1, var1, mean2, var2, quantiles1,quantiles2

In [9]:
# fit ensemble
start_time = time.time()
n_members = 10
ensemble = fit_ensemble(n_members, X_train_ensemble, y_train_ensemble)

# train data mean and variance vectors

mean_vec1 = list()
mean_vec2 = list()
var_vec1 = list()
var_vec2 = list()

for i in range(X_train_mse.shape[0]):
    newX = np.asarray([X_train_mse[i, :]])
    mean1,var1,mean2,var2,_ ,__= predict_with_pi(ensemble, newX)
    mean_vec1.append(mean1)
    mean_vec2.append(mean2)
    var_vec1.append(var1)
    var_vec2.append(var2)
    print(i)
end_time = time.time()
print("%s seconds", end_time - start_time)

In [13]:
# random error calculation on training data
r1 = list()
r2 = list()
r1 = (y_train_mse[:,0] - mean_vec1)**2 - var_vec1
for i in range(len(r1)):
    if r1[i] < 0: r1[i] = 0.0
r2 = (y_train_mse[:,1] - mean_vec2)**2 - var_vec2
for i in range(len(r2)):
    if r2[i] < 0: r2[i] = 0.0

In [14]:
def calc_distance(s1,s2):
    return(np.sqrt((s1[0] - s2[0])**2 + (s1[1] - s2[1])**2))

def get_nearest_data(s_train,s_test,r,k):
    dist_mat = np.zeros((len(s_test),len(s_train)))
    nearest_var = np.zeros((len(s_test)))
    for i in range(len(s_train)):
        for j in range(len(s_test)):
            dist_mat[j][i] = calc_distance(s_train[i],s_test[j])
    for i in range(len(s_test)):
        a = dist_mat[i]
        b = np.argpartition(a,k)[:k]
        y_val = []
        for index in b:
            y_val.append(r[index])
        nearest_var[i] = (np.mean(y_val))
    return nearest_var

In [1]:
### Test data 

df_test = pd.read_csv("../test_data_nonstationary.csv", sep = ",")
s_test = np.vstack((df_test["x"],df_test["y"])).T
y_test = np.array(df_test[["var1","var2"]])

# mean and variance vector for prediction data
N = len(df_test)
s = s_test
num_basis = [2**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
#knots_1d = [np.linspace(0,1,i) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]
encoder_test = phi

NameError: name 'pd' is not defined

In [36]:
# random error calculation on test data with neighbourhood approach
r1_pred = get_nearest_data(s_train_mse,s_test,r1,10)
r2_pred = get_nearest_data(s_train_mse,s_test,r2,10)

In [37]:
# test data bounds 
mean_vec1 = list()
mean_vec2 = list()
var_vec1 = list()
var_vec2 = list()
l_quantile1 = list()
u_quantile1 = list()
l_quantile2 = list()
u_quantile2 = list()
for i in range(encoder_test.shape[0]):
    newX = np.asarray([encoder_test[i, :]])
    mean1,var1,mean2,var2,quantile1,quantile2 = predict_with_pi(ensemble, newX)
    mean_vec1.append(mean1)
    mean_vec2.append(mean2)
    var_vec1.append(var1)
    var_vec2.append(var2)
    l_quantile1.append(quantile1[0])
    u_quantile1.append(quantile1[1])
    l_quantile2.append(quantile2[0])
    u_quantile2.append(quantile2[1])
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [38]:
# e = list()
# for i in range(len(r1_pred)):
#     e.append(r1_pred[i][0])
# print(len(var_vec1))
# error_vec2 = model_mse2.predict(encoder_test)
var_vec1 = np.asarray(var_vec1)
var_vec2 = np.asarray(var_vec2)
# e = np.asarray(e)
lower_bound1 = mean_vec1 - 1.96*np.sqrt(var_vec1 + r1_pred)
upper_bound1 = mean_vec1 + 1.96*np.sqrt(var_vec1 + r1_pred)
lower_bound2 = mean_vec2 - 1.96*np.sqrt(var_vec2 + r2_pred)
upper_bound2 = mean_vec2 + 1.96*np.sqrt(var_vec2 + r2_pred)

In [32]:
# Training case
count_var0 = 0
count_var1 = 0
for i in range(len(y_test)):
    if ((y_test[i,0] < lower_bound1[i]) or (y_test[i,0] > upper_bound1[i])): count_var0 +=1
    if y_test[i,1] < lower_bound2[i] or y_test[i,1] > upper_bound2[i]: count_var1 +=1
picp1 = count_var0/len(y_test)
picp2 = count_var1/len(y_test)
print(str(picp1)+","+str(picp2))
width1 = np.mean(upper_bound1 - lower_bound1)
width2 = np.mean(upper_bound2 - lower_bound2)
print(str(width1)+","+str(width2))

0.05416666666666667,0.04583333333333333
0.13623351655125943,0.1863167077162933


In [43]:
# Test data case
count_var0 = 0
count_var1 = 0
for i in range(len(y_test)):
    if ((y_test[i,0] < lower_bound1[i]) or (y_test[i,0] > upper_bound1[i])): count_var0 +=1
    if y_test[i,1] < lower_bound2[i] or y_test[i,1] > upper_bound2[i]: count_var1 +=1
picp1 = count_var0/len(y_test)
picp2 = count_var1/len(y_test)
print(str(picp1)+","+str(picp2))
width1 = np.mean(upper_bound1 - lower_bound1)
width2 = np.mean(upper_bound2 - lower_bound2)
print(str(width1)+","+str(width2))

0.0125,0.02
0.12593444431744494,0.17568411271396628


In [44]:
df_bounds = pd.DataFrame(np.vstack((lower_bound1,upper_bound1,lower_bound2,upper_bound2)).T,
                         columns = ["l1","u1","l2","u2"])
df_bounds.to_csv("../plot_results/bounds_nonstationary.csv")